## THIS WILL NOT WORK IN GOOGLE COLAB! PLEASE EXPORT AS AN .IPYNB FILE AND RUN LOCALLY!

# MongoDB

### Super quick JSON review

In [42]:
!pip install pymongo
import pymongo
from pymongo import MongoClient
import pandas as pd

In [43]:
client = MongoClient('localhost', 27017)
client
db = client.example
collection = db.music

### Setting things up

Remember to restart the kernel after running this code cell if you didn't already have pymongo installed (you only need to do this once)

In [26]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [61]:
songs = pd.read_csv('/Users/ellathomasson/Documents/Jupiter Notebook/Project 2 - Songs (1).csv')
artists = pd.read_csv('/Users/ellathomasson/Documents/Jupiter Notebook/Project 2 - Artists.csv')
albums = pd.read_csv('/Users/ellathomasson/Documents/Jupiter Notebook/Project 2 - Albums (3).csv')
playlists = pd.read_csv('/Users/ellathomasson/Documents/Jupiter Notebook/Project 2 - Playlists.csv')
users = pd.read_csv('/Users/ellathomasson/Documents/Jupiter Notebook/Project 2 - Users (1).csv')
users

,Creator_ID,Username,Country,Top_Song_ID,Top_Album_ID,Top_Artist_ID,Best_friend_ID
0,1,Blink21,USA,9.0,2.0,8.0,3.0
1,2,JaxWave,USA,NaN,NaN,NaN,4.0
2,3,LunaPix,Canada,16.0,9.0,2.0,6.0
3,4,ZorbX,USA,NaN,NaN,NaN,7.0
4,5,FizzDart,Mexico,34.0,2.0,6.0,NaN
5,6,QuikFox,Canada,9.0,2.0,8.0,3.0
6,7,HexRyn,Sweden,19.0,NaN,1.0,8.0
7,8,VimJet,Ireland,1.0,1.0,1.0,2.0
8,9,KyroM,USA,36.0,2.0,8.0,8.0
9,10,WispFiz,USA,54.0,11.0,15.0,6.0


In [62]:
merged_df = pd.merge(playlists, songs, on='Playlist_ID', how='left')
playlist_songs = merged_df.groupby('Playlist_ID')['Title'].apply(list).reset_index()
playlists_df = pd.merge(playlists, playlist_songs, on='Playlist_ID', how='left')
playlists_df['Title'] = playlists_df['Title'].apply(lambda x: x if isinstance(x, list) else [])
playlists_df = playlists_df.rename(columns={'Title': 'Songs_in_playlist'})
print(playlists_df.head(5))

   Playlist_ID  Playlist_name  Creator_ID                    Description  \
0            1   Driving Time           1             Songs to drive to.   
1            2            Gym           2             Songs for the gym.   
2            3   Cooking Time           4        Songs for cooking time.   
3            4          Happy           3  When you have had a GREAT day   
4            5  cleaning time           6                    bops fr fr    

   Popularity                                  Songs_in_playlist  
0        1000  [Happier, Golden , Love to Dream, Yellow, Gold...  
1        2500  [White Horse, High Hopes, Watermelon Sugar, Sh...  
2        1200  [Two-Headed Boy, Vampire, Love Like This, All ...  
3           0  [Enough for You, When We Were Young , Baby for...  
4           1  [Kiss Me More, Adore You, All Too Well, Love S...  


In [47]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [64]:
users_dict = users.to_dict('records')
users_dict
songs_dict = songs.to_dict('records')
albums_dict = albums.to_dict('records')
playlists_dict = playlists_df.to_dict('records')
artists_dict = artists.to_dict('records')

In [65]:
users_collection = db.users
albums_collection = db.albums
artists_collection = db.artists
playlists_collection = db.playlists
songs_collection = db.songs

### Inserting/deleting data

MongoDB is made up of, in a simplified way, massive JSON objects

In [66]:
users_collection.insert_many(users_dict)
albums_collection.insert_many(albums_dict)
songs_collection.insert_many(songs_dict)
artists_collection.insert_many(artists_dict)
playlists_collection.insert_many(playlists_dict)

InsertManyResult([ObjectId('665738c1eaf5525eb07520df'), ObjectId('665738c1eaf5525eb07520e0'), ObjectId('665738c1eaf5525eb07520e1'), ObjectId('665738c1eaf5525eb07520e2'), ObjectId('665738c1eaf5525eb07520e3'), ObjectId('665738c1eaf5525eb07520e4'), ObjectId('665738c1eaf5525eb07520e5'), ObjectId('665738c1eaf5525eb07520e6'), ObjectId('665738c1eaf5525eb07520e7'), ObjectId('665738c1eaf5525eb07520e8')], acknowledged=True)

In [69]:
  SELECT users.Username, users.Country, songs.Title AS Fav_Song, Artist_Country, songs.Rating
  FROM users
  LEFT JOIN songs ON users.Top_Song_ID = songs.Song_ID
  LEFT JOIN artists ON songs.Artist_ID = artists.Artist_ID
  WHERE users.Country <> artists.Artist_Country
  ORDER BY songs.Rating DESC


In [84]:
try:
    pipeline = [
        {
            "$lookup": {
                "from": "songs_collection",
                "localField": "Top_Song_ID",
                "foreignField": "Song_ID",
                "as": "fav_song"
            }
        },
        {
            "$unwind": {
                "path": "$fav_song",
                "preserveNullAndEmptyArrays": True
            }
        },
        {
            "$lookup": {
                "from": "artists_collection",
                "localField": "fav_song.Artist_ID",
                "foreignField": "Artist_ID",
                "as": "artist"
            }
        },
        {
            "$unwind": {
                "path": "$artist",
                "preserveNullAndEmptyArrays": True
            }
        },
        {
            "$match": {
                "$expr": {
                    "$ne": ["$Country", "$artist.Artist_Country"]
                }
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the default _id field
                "Username": 1,
                "Country": 1,
                "Fav_Song": "$fav_song.Title",
                "Artist_Country": "$artist.Artist_Country",
                "Rating": "$fav_song.Rating"
            }
        },
        {
            "$sort": {"Rating": -1}
        }
    ]
    
    result = db.users_collection.aggregate(pipeline)

    for doc in result:
        print(doc)

except Exception as e:
    print("An error occurred:", str(e))


In [91]:

pipeline = [
    {
        "$group": {
            "_id": "$Genre",
            "average_rating": {"$avg": "$Rating"}
        }
    }
]


results = songs_collection.aggregate(pipeline)
df = pd.DataFrame(list(results))
df.head()


,_id,average_rating
0,Pop,7.805556
1,Rock,8.000000
2,Indie,6.846154
3,Bollywood,9.000000
4,Alternative Rock,8.000000


InsertManyResult([ObjectId('665735c5eaf5525eb0752079'), ObjectId('665735c5eaf5525eb075207a'), ObjectId('665735c5eaf5525eb075207b')], acknowledged=True)

DeleteResult({'n': 2, 'ok': 1.0}, acknowledged=True)

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

### Querying data

Let's explore some example data that might be similar to the type you are creating in your projects

In [73]:
employees = pd.read_csv('https://raw.githubusercontent.com/hmf9kx/LDS2022/main/week2/employees.csv')
employees

,employee_id,first_name,last_name,age,job_title,team,email,phone_number
0,1,Adam,Anderson,34,Sales Manager,A,adama@gmail.com,4341728463
1,2,Barbara,Blake,75,Sales Manager,B,barbarab@gmail.com,4342617564
2,3,Cody,Covington,24,Sales Representative,A,codyc@gmail.com,4342672819
3,4,Dave,Douglass,54,Sales Representative,B,daved@gmail.com,4341675830
4,5,Evan,Earhart,43,Sales Representative,A,evane@gmail.com,4341529805
5,6,Frank,Falconer,23,Sales Representative,B,frankf@gmail.com,4341678923
6,7,Grace,Gaetz,27,Sales Representative,A,graceg@gmail.com,4347869041
7,8,Hailey,Harrington,64,Sales Representative,B,haileyh@gmail.com,4345618953
8,9,Igor,Irvin,43,Sales Representative,A,igori@gmail.com,4347759109
9,10,James,Johnson,40,Sales Representative,B,jamesj@gmail.com,4347859016


We can easily convert flat data to dictionary format with `to_dict()`

In [33]:
employees_dict = employees.to_dict('records')
employees_dict

[{'employee_id': 1,
  'first_name': 'Adam',
  'last_name': 'Anderson',
  'age': 34,
  'job_title': 'Sales Manager',
  'team': 'A',
  'email': 'adama@gmail.com',
  'phone_number': 4341728463},
 {'employee_id': 2,
  'first_name': 'Barbara',
  'last_name': 'Blake',
  'age': 75,
  'job_title': 'Sales Manager',
  'team': 'B',
  'email': 'barbarab@gmail.com',
  'phone_number': 4342617564},
 {'employee_id': 3,
  'first_name': 'Cody',
  'last_name': 'Covington',
  'age': 24,
  'job_title': 'Sales Representative',
  'team': 'A',
  'email': 'codyc@gmail.com',
  'phone_number': 4342672819},
 {'employee_id': 4,
  'first_name': 'Dave',
  'last_name': 'Douglass',
  'age': 54,
  'job_title': 'Sales Representative',
  'team': 'B',
  'email': 'daved@gmail.com',
  'phone_number': 4341675830},
 {'employee_id': 5,
  'first_name': 'Evan',
  'last_name': 'Earhart',
  'age': 43,
  'job_title': 'Sales Representative',
  'team': 'A',
  'email': 'evane@gmail.com',
  'phone_number': 4341529805},
 {'employee_id':

We can then insert it into our MongoDB

In [36]:
employee_collection = db.employees

In [37]:
employee_collection.insert_many(employees_dict)

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 6656287c31419e49de8447d6, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

Just like how we create data with dictionaries, we query data the same way

In [ ]:
query = {'first_name': 'Adam'}
employee_collection.find_one(query)

In [ ]:
query = {'team': 'A'}
employee_collection.find(query)

In [ ]:
query = {'team': 'A'}
result = employee_collection.find(query)

In [ ]:
for e in result:
    print(e)

We can use prettyprint to make things a bit more human readable

In [ ]:
import pprint

In [ ]:
query = {'team': 'A'}
result = employee_collection.find(query)

In [ ]:
for e in result:
    pprint.pprint(e)

In [ ]:
age_query = {'age': {'$lt': 30}}
result = employee_collection.find(age_query)

In [ ]:
for e in result:
    pprint.pprint(e)

We can also choose to only output information that we need, through selecting columns

In [ ]:
for e in result:
    pprint.pprint(e['first_name'])

For these test databases, it is good practice to delete once you are done, so you do not take up space on your computer

In [38]:
employee_collection.delete_many({})

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 6656287c31419e49de8447d6, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>